In [132]:
from __future__ import print_function
import sys
sys.path.append('../build/')
%pylab inline
np.set_printoptions(precision=4, suppress=True)
import versor as vsr
from versor.drawing import *

Populating the interactive namespace from numpy and matplotlib


/Users/lars/anaconda3/lib/python3.5/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['Inf']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [177]:
zero = vsr.CGA(*[0]*32)
one = vsr.CGA(*[0]*32); one[0] = 1
ni = vsr.CGA(*[0]*32); ni[5] = 1
no = vsr.CGA(*[0]*32); no[4] = 1
E = no ^ ni
e1 = vsr.CGA(vsr.Vec(1,0,0))
e2 = vsr.CGA(vsr.Vec(0,1,0))
e3 = vsr.CGA(vsr.Vec(0,0,1))
I3 = e1 * e2 * e3
I = no * I3 * ni
epsr = I3 * no
e1nir= (-1)**1 * (e2 ^ e3 ^ no) <= I.inv()
e2nir= (-1)**2 * (e1 ^ e3 ^ no) <= I.inv()
e3nir= (-1)**3 * (e1 ^ e2 ^ no) <= I.inv()

In [180]:
e1r = (-1)**0 * (e2 ^ e3 ^ no ^ ni) <= I.inv()
e2r = (-1)**1 * (e1 ^ e3 ^ no ^ ni) <= I.inv()
e3r = (-1)**2 * (e1 ^ e2 ^ no ^ ni) <= I.inv()
nor = (-1)**3 * (e1 ^ e2 ^ e3 ^ ni) <= I.inv()
nir = (-1)**4 * (e1 ^ e2 ^ e3 ^ no) <= I.inv()
e1r + e2r + e3r + nor + nir

CGA: [ 0 1 1 1 -1 -1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ]

In [181]:
def cga_reciprocal_line(l):
    A = l.biv()
    b = l.drv().vec()
    return vsr.CGA(A.rev()) + b[0] * e1nir + b[1] * e2nir + b[2] * e3nir

In [182]:
def cga_reciprocal_motor(m):
    s = m[0] * one
    A = m[1] * e2 * e1 + m[2] * e3 * e1 + m[3] * e3 * e2
    b = m[4] * e1nir + m[5] * e2nir + m[6] * e3nir
    ps = m[7] * I3.rev() * no    
    mr = s + A + b + ps
    return mr

In [183]:
cga_motor

CGA: [ 0.87 0 0 0 0 0 -0.13 -0.27 -0.4 0 0 0 -0.16 -0.22 -0.12 0 0 0 0 0 0 0 0 0 0 0 0 0.021 0 0 0 0 ]

In [184]:
cga_reciprocal_motor(motor)

CGA: [ 0.87 0 0 0 0 0 0.13 0.27 0.4 -0.16 -0.22 -0.12 0 0 0 0 0 0 0 0 0 0 0 0 0 0 -0.021 0 0 0 0 0 ]

In [185]:
cga_reciprocal_motor(motor) * cga_l1r * cga_reciprocal_motor(motor).rev()

CGA: [ 0 0 0 0 0 0 0.36 -0.45 -0.82 0.17 -0.11 0.54 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ]

In [186]:
cga_motor * cga_l1 * cga_motor.rev()

CGA: [ 0 0 0 0 0 0 0.57 -0.017 0.82 0 0 0 0.17 0.3 -0.26 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ]

In [187]:
def cga_squared_norm(a,b):
    return (a * b)[0]

In [188]:
delta = 0.6
theta = pi/6
motor = (vsr.Vec(1,2,3).unit() * delta ).trs() * vsr.Rot(vsr.Biv(1,2,3).unit() * theta)
cga_motor = vsr.CGA(motor)
cga_motor_r = cga_reciprocal_motor(motor)

In [211]:
l1 = vsr.Dll(vsr.Vec(0,0,0).null(), vsr.Vec(1,0,0).unit())
cga_l1r = cga_reciprocal_line(l1)
cga_l1 = vsr.CGA(l1)
l2 = l1.spin(motor)
cga_l2r = cga_reciprocal_line(l2)
cga_l2 = vsr.CGA(l2)

In [221]:
m = motor

In [222]:
e12

NameError: name 'e12' is not defined

In [227]:
M = np.array([np.array(vsr.Mot(m[0],0,0,0,0,0,0,0,) * l1),
             np.array(vsr.Mot(0,m[1],0,0,0,0,0,0,) * l1),
                np.array(vsr.Mot(0,0,m[2],0,0,0,0,0,) * l1),
                   np.array(vsr.Mot(0,0,0,m[3],0,0,0,0,) * l1),
                      np.array(vsr.Mot(0,0,0,0,m[4],0,0,0,) * l1),
                         np.array(vsr.Mot(0,0,0,0,0,m[5],0,0,) * l1),
                            np.array(vsr.Mot(0,0,0,0,0,0,m[6],0) * l1),
                               np.array(vsr.Mot(0,0,0,0,0,0,0,m[7]) * l1)])

In [231]:
np.dot(M.T, motor.rev())

array([ 0.1607,  0.0714, -0.0179,  0.75  , -0.0005,  0.015 , -0.0504,
       -0.0241])

In [232]:
l2

Dll: [ 0.5701 -0.01717 0.8214 0.1746 0.3038 -0.2607 ]

In [190]:
cga_Sd = cga_l2 - cga_l1; print(cga_S)
cga_Sdr = cga_l2r - cga_l1r; print(cga_Sr)
print(cga_reciprocal_line(l2 - l1))

CGA: [ 0 0 0 0 0 0 0.57 -0.017 -0.18 0 0 0 0.17 0.3 -0.26 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ]
CGA: [ 0 0 0 0 0 0 -0.57 0.017 0.18 0.17 0.3 -0.26 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ]
CGA: [ 0 0 0 0 0 0 -0.57 0.017 0.18 0.17 0.3 -0.26 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ]


In [192]:
print(cga_l1 <= cga_l2r)

CGA: [ 0.82 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ]


In [196]:
cga_l2 <= cga_l2r

CGA: [ 1.2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ]

In [199]:
(cga_l2 - cga_l1)<=(cga_l2r - cga_l1r)

CGA: [ 0.55 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ]

In [195]:
cga_l1 * cga_l1r - cga_l1 * cga_l2r - cga_l2 * cga_l1r + cga_l2 * cga_l2r

CGA: [ 0.55 0 0 0 0 0 0 0 0 0.18 -0.053 0.057 0.18 -0.053 0.057 0.19 0 0 0 0 0 0 0 0 0 0 -0.17 0.17 0 0 0 0 ]

In [201]:
cga_l2<=cga_l2r

CGA: [ 1.2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ]

In [210]:
cga_l2 <=  reciprocal_line((motor * l1 * motor.rev()).dll())

TypeError: __le__(): incompatible function arguments. The following argument types are supported:
    1. (self: versor_pybind11.CGA, arg0: versor_pybind11.CGA) -> versor_pybind11.CGA

Invoked with: CGA: [ 0 0 0 0 0 0 0.57 -0.017 0.82 0 0 0 0.17 0.3 -0.26 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ], None

In [129]:
(cga_l1 * cga_l2r *  -1.0) - cga_l2 * cga_l1r

CGA: [ -1.6 -0 -0 -0 -0 -0 0 0 -0 -0 0.26 0.3 -0 0.26 0.3 -0 -0 -0 -0 -0 -0 -0 -0 -0 -0 -0 -0.17 0.17 -0 -0 -0 -0 ]

In [130]:
(cga_Sd * cga_Sdr)[0]

0.547840622407381

In [127]:
np.inner(cga_Sd, cga_Sd)

0.54784062240738096

In [128]:
np.inner(l2-l1, l2-l1)

0.54784062240738096

In [156]:
cga_Sc = vsr.CGA(l2.comm(l1.rev())); print(cga_Sc)
cga_Scr = cga_reciprocal_line(l2.comm(l1.rev())); print(cga_Scr)

CGA: [ 0 0 0 0 0 0 -0.017 -0.57 0 0 0 0 0 -0.26 -0.3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ]
CGA: [ 0 0 0 0 0 0 0.017 0.57 0 0 -0.26 -0.3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ]


In [157]:
cga_Sc <= cga_Scr

CGA: [ 0.49 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ]

In [158]:
np.inner(l2.comm(l1.rev()),l2.comm(l1.rev()))

0.48548027261980015

In [159]:
cga_reciprocal_line(l2) * vsr.CGA(l2)

CGA: [ 1.2 0 0 0 0 0 0 0 0 -0.18 0.31 0.25 -0.18 0.31 0.25 -0.19 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ]

In [160]:
np.inner(l2,l2)

1.1906977652645241

In [161]:
cga_reciprocal_line(l1) * vsr.CGA(l1)

CGA: [ 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ]

In [162]:
cga_l1.spin(cga_motor)

CGA: [ 0 0 0 0 0 0 0.57 -0.017 0.82 0 0 0 0.17 0.3 -0.26 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ]

In [165]:
cga_l1r.spin(cga_motor_r)

CGA: [ 0 0 0 0 0 0 0.36 -0.45 -0.82 0.17 -0.11 0.54 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ]